In [ ]:
from flask import Flask #flask에서 Flask라는 클래스 import
import urllib.request

app = Flask(__name__) #Flask라는 클래스의 객체를 생성(app), 인수: __name__

@app.route("/tospring", methods=['GET']) #생성한 객체의 route 설정(=URL 설정)
def spring():
    return marko_json

if __name__ == '__main__': #객체의 run함수를 이용해서 로컬 서버에서 어플리케이션 실행
    app.run(debug=False, host="127.0.0.1", port=5000)

In [4]:
# !pip install import_ipynb
# !pip install pykrx
# !pip install -U finance-datareader
# !pip install openpyxl
# !pip install pyyaml

# # Markowitz
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

from datetime import datetime, timedelta, date
from pykrx import stock

import FinanceDataReader as web

# # Auto Trading & Alert
# import requests
# import time
# import yaml

# # Recommend
import getpass
import openpyxl
from urllib.request import urlopen

import import_ipynb

In [5]:
n = 1
num_stock = 3
stock_name = ["삼성전자", "카카오", "대한항공"]

In [7]:
n = int(input("숫자 입력(0: Recommend, 1: Markowitz, 2: Vol_auto, 3: Port_auto, 4: stock_alert): "))
if n == 0:
    # 종목코드
    stock_code ="";

    stock_file = pd.read_excel('C:/Users/yeeju/Desktop/Codes/markowitz_src/종목전체검색.xlsx', engine='openpyxl')

    B1=stock_file.loc[stock_file.Name == stock_name, ['b1']]
    list = B1['b1'].values.tolist()
    
    if list[0] == 'IT': 
        list_mid = ['디스플레이', '반도체', '소프트웨어', '하드웨어']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_IT.xlsx', engine='openpyxl')

    elif list[0] == '경기소비재':
        list_mid = ['내구 소비재 및 의류', '미디어', '소비자 서비스', '유통', '자동차 및 부품']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_com.xlsx', engine='openpyxl')

    elif list[0] == '금융':
        list_mid = ['기타금융', '보험', '부동산', '은행', '증권']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_bank.xlsx', engine='openpyxl')

    elif list[0] == '산업재':
        list_mid = ['상업서비스', '운송', '자본재']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_industrial.xlsx', engine='openpyxl')

    elif list[0] == '소재':
        list_mid = ['소재']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_Material.xlsx', engine='openpyxl')

    elif list[0] == '에너지':
        list_mid = ['에너지']
        stock= pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_energy.xlsx', engine='openpyxl')

    elif list[0] == '유틸리티':
        list_mid = ['유틸리티']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_utility.xlsx', engine='openpyxl')

    elif list[0] == '의료':
        list_mid = ['의료장비 및 서비스','제약 및 바이오']
        stock = pd.read_excel('/Users/yeeju/Desktop/Codes/markowitz_src/result_medical.xlsx', engine='openpyxl')

    else:
        list_mid = ['생활용품', '음식료 및 담배']
        
    # 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
    stock.Code = stock.Code.map('{:06d}'.format) #6자리가 아닌 수를 앞에 0으로 채우기 위함
    
    # 한국거래소에 상장된 모든 종목 리스트 가져오기
    df_krx = web.StockListing('KRX')
    
    #어제, 오늘 날짜를 간단히 구하는 방법
    today = date.today()
    yesterday = date.today() - timedelta(1)
    end= today.strftime('%Y-%m-%d')
    start= yesterday.strftime('%Y-%m-%d')
    
    #변동성 계산
    def cal_vix(open1, High, Low):

        vix = open1 +(High-Low)*0.5

        return vix
    
    #종목명
    stock_name = stock["Name"]
    
    #대분류한 종목 코드 열 추출
    stock_c = stock["Code"]
    
    #종목 명 열 추출 한것 리스트로 변환
    stock_clist = stock_c.values.tolist()
    
    #종목 코드 열 추출 한것 리스트로 변환
    stock_name_list = stock_name.values.tolist()
    
    #대분류한 중분류  열 추출
    stock_b2 = stock["b2"]
    
    #종목 코드 열 추출 한것 리스트로 변환
    stock_b2_list = stock_b2.values.tolist()
    
    item_list=[]#시가,고가,종가 출력하는 코드
    for item_code in stock_clist:
        data_price = web.DataReader(str(item_code), start, end)
        item_list.append(data_price)
    
    #전날, 당일 추출 
    OpenP_all_list=[]
    OpenH_all_list=[]
    OpenL_all_list=[]
    for i in range(len(item_list)):
            #시가
            stock_open = item_list[i].Open
            OpenP_all_list.append(stock_open)
            #고가
            stock_High = item_list[i].High
            OpenH_all_list.append(stock_High)
            #저가
            stock_Low = item_list[i].Low
            OpenL_all_list.append(stock_Low)
    
    OpenP_list=[]
    OpenH_list=[]
    OpenL_list=[]
    vix_list=[]
    for i in range(len(item_list)):
        #당일시가 추출
        OpenP = OpenP_all_list[i][1]
        OpenP_list.append(OpenP)
        #전날 고가 추출
        OpenH = OpenH_all_list[i][0]
        OpenH_list.append(OpenH)
        #전날 저가 추출
        OpenL = OpenL_all_list[i][0]
        OpenL_list.append(OpenL)
        Vix = cal_vix(OpenP_list[i], OpenH_list[i], OpenL_list[i])
        #print(Vix)
        vix_list.append(Vix)
    
    #중분류, 변동성 매칭
    vix_b2_list=[]
    for i in range(len(stock_c)):
        list = [stock_name_list[i],vix_list[i], stock_b2_list[i]]
        vix_b2_list.append(list)
    
    dis_list_0 = []
    dis_list_1 = []
    dis_list_2 = []
    dis_list_3 = []
    dis_list_4 = []

    for i in range(len(list_mid)):
        for j in range(len(stock_c)):
            if list_mid[i] == vix_b2_list[j][2]:
                globals()['dis_list_'+str(i)].append(vix_b2_list[j])
    
    def Top1(L) :
        L.sort()
        return L[-1]
    
    def Top2(L) :
        L.sort()
        return L[-2]
    
    def Top3(L) :
        L.sort()
        return L[-3]
    
    Top3_0 = []
    Top3_1 = []
    Top3_2 = []
    Top3_3 = []
    Top3_4 = []

    for i in range(len(list_mid)):
        a = Top1(globals()['dis_list_'+str(i)])
        globals()['Top3_'+str(i)].append(a)
        b = Top2(globals()['dis_list_'+str(i)])
        globals()['Top3_'+str(i)].append(b)
        c= Top3(globals()['dis_list_'+str(i)])
        globals()['Top3_'+str(i)].append(c)
    
    Top3_name_0 =[]
    Top3_name_1 =[]
    Top3_name_2 =[]
    Top3_name_3 =[]
    Top3_name_4 =[]
    for i in range(len(list_mid)):
        for j in range(len(Top3_1)):
            sn = globals()['Top3_'+str(i)][j][0]
            globals()['Top3_name_'+str(i)].append(sn)
    
    for i in range(len(list_mid)):
        globals()['stc_name_'+str(i)] = list_mid[i]
    
    dict_list =[]
    for i in range(len(list_mid)):
        a = {list_mid[i]: globals()['Top3_name_'+str(i)]}
        dict_list.append(a)
    
    recom_dict = {"mid_recommend": dict_list} 
    
    recommend_json = json.dumps(recom_dict, indent=4,ensure_ascii=False)
    
    print(recommend_json)
elif n == 1:
    # 데이터 가져오기

    def stock_reader(kospi_df, stock_name, code_list, df='p', n=0):

        if n == 0:
            n = len(code_list)

        stock_df = pd.DataFrame()
        stock_df['Date'] = kospi_df['Date']
        print("동기간 KOSPI 생성일수 : ", len(kospi_df['Date']))
        normal_cnt = 0
        err_cnt = 0
        code_nm_list = []
        symbol_list = []

        print("n", n)

        for code in code_list:
            stock = df_krx[df_krx.Code == code]
            code_nm = list(stock.Name)[0]
            symbol = list(stock.Code)[0]


            try:
                temp = web.DataReader(code, start_date, end_date)

                # 데이터일수가 시장보다 작으면 skip(최근 상장 데이터로 판단)
                if len(temp) < len(kospi_df['Date']):
                    err_cnt += 1
                    print("skip : (",err_cnt,")", code_list[code], code_nm, start_date, end_date, ", 건수 : ", len(temp))
                    continue

                temp.reset_index(inplace = True)
                temp_df = pd.merge(temp[['Date','Close', 'Open']], kospi_df[['Date']], on='Date', how='right')

                if df == 'v':
                    stock_df[code_nm] = temp_df.Close - temp_df.Open #variation
                elif df == 'p':
                    stock_df[code_nm] = temp_df.Close # price

                normal_cnt += 1
                code_nm_list.append(code_nm)
                symbol_list.append(symbol)
                print("정상 : (",normal_cnt,")", code, code_nm, start_date, end_date, ", 건수 : ", len(temp), "->", len(stock_df))
            except:
                err_cnt += 1
                print("skip : (",err_cnt,")", code, code_nm, start_date, end_date, ", 건수 : ", len(temp))

            if normal_cnt == n:
                print('총', n,'개 생성 설정 / ', normal_cnt, '개 생성 완료')
                break # n개 종목 생성시 종료

            # 데이터 정렬
            stock_df.sort_values('Date', ascending=True, inplace=True) # ascending=True 오름차순, False 내림차순
            # 결측치 채우기
            stock_df.fillna(method='ffill', inplace=True)

        return stock_df, code_nm_list, symbol_list

    code_list = [] # 종목 코드 리스트
    tickers = stock.get_market_ticker_list(market="KOSPI")
    for i in range(num_stock):
        for ticker in tickers:
            if stock_name[i] == stock.get_market_ticker_name(ticker):
                code_list.append(ticker)
    
    start_date = date.today() - timedelta(days=366)
    end_date = date.today() - timedelta(days=1)

    # 한국 거래소 상장종목 전체
    # 용도 : 코드와 종목명 가져오기
    df_krx = web.StockListing('KRX')

    kospi_df = web.DataReader('KS11', start_date, end_date)
    kospi_df.reset_index(inplace = True)
    
    stock_df, code_nm_list, symbol_list = stock_reader(kospi_df, stock_name, code_list, 'p', num_stock) # v :variation, p : price(close)
    
    stock_df.dropna(inplace=True)
    data = stock_df.set_index('Date')
    
    # 현재 포트폴리오 설정 (랜덤 Weight로 산출)
    num_assets = np.size(code_nm_list)
    cur_value = (1e4-5e3)*np.random.rand(num_assets,1) + 5e3        
    tot_value = np.sum(cur_value)
    weights = cur_value.ravel()/float(tot_value)
    
    #compute portfolio risk
    Sigma = data.cov().values
    Corr = data.corr().values

    volatility = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
    
    weight_list = weights.tolist()

    marko_dict = {"markowitz":{"stk_nm":num_stock, "stk_cd":code_list, "stk_wt":weight_list}}
    
    marko_json = json.dumps(marko_dict, indent=4)
    
    print(marko_json)
elif n == 2:
    import VolatilityStockAutoTrade as V
    print(V.VAT_json)
elif n == 3:
    import PortfolioStockAutoTrade as P
    print(P.PAT_json)
elif n == 4:
    import stock_alert as A
    print(A.alert_json)
else:
    print("에러!")

숫자 입력(0: Recommend, 1: Markowitz, 2: Vol_auto, 3: Port_auto, 4: stock_alert): 1
동기간 KOSPI 생성일수 :  245
n 3
정상 : ( 1 ) 005930 삼성전자 2021-11-15 2022-11-15 , 건수 :  248 -> 245
정상 : ( 2 ) 035720 카카오 2021-11-15 2022-11-15 , 건수 :  248 -> 245
정상 : ( 3 ) 003490 대한항공 2021-11-15 2022-11-15 , 건수 :  248 -> 245
총 3 개 생성 설정 /  3 개 생성 완료
{
    "markowitz": {
        "stk_nm": 3,
        "stk_cd": [
            "005930",
            "035720",
            "003490"
        ],
        "stk_wt": [
            0.4171597347539247,
            0.2991216199095051,
            0.28371864533657026
        ]
    }
}


In [ ]:
app = Flask(__name__) #Flask라는 클래스의 객체를 생성(app), 인수: __name__

@app.route("/tospring", methods=['POST']) #생성한 객체의 route 설정(=URL 설정)
def spring():
    return marko_json

if __name__ == '__main__': #객체의 run함수를 이용해서 로컬 서버에서 어플리케이션 실행
    app.run(debug=False, host="127.0.0.1", port=5000)